In [20]:
# Define the tool functions
@tool
def square_number_tool(number: float) -> float:
    """
    calculate the square of a number
    """
    return number ** 2

@tool
def get_stock_df(ticker_symbol:str, start_date:str, end_date:str,interval:str="1d", columns_needed:Set[str]=set({"Close","Volume"}))->pd.DataFrame:
    """get the prices of a stock ticker between 2 days"""
    MOVING_AVG = f"{ticker_symbol}_M_A"
    moving_average_window = {"1d": 5, "5d": 4, "1wk": 4, "1mo": 3, "3mo": 2}.get(interval) or 5
    stock = yf.Ticker(ticker_symbol)
    stock_history_df = stock.history(start=start_date, end=end_date,interval=interval)
    stock_history_df[MOVING_AVG] = stock_history_df["Close"].rolling(window=moving_average_window).mean()
    stock_history_df = stock_history_df[list(columns_needed)+[MOVING_AVG]]
    stock_history_df.rename(columns={column_name:f"{ticker_symbol}_{column_name[:3]}" for column_name in columns_needed}, inplace=True)
    return stock_history_df



In [21]:
# Initialize the LLM and memory
llm = ChatOpenAI(
    temperature=0.4,
    model='gpt-3.5-turbo-1106'
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly assistant called Max. You are bad at math calculation, use tool when you need to do arithmetic"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

tools = [square_number_tool, get_stock_df]
agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

user_input = "Get the stock price and moving average of MSFT between 2024-10-10 to 2024-10-30 then square the numbers"

response = agent_executor.invoke({
        "input": user_input,
    })
# the model choose to report closing price
print(response["output"])

The stock prices and moving averages for MSFT between 2024-10-10 and 2024-10-30, along with their squares, are as follows:

| Date       | Close Price | Moving Average | Close Price (Squared) |
|------------|-------------|----------------|------------------------|
| 2024-10-10 | 415.839996  | NaN            | 172922.90227328        |
| 2024-10-11 | 416.320007  | NaN            | 173322.34822848003     |
| 2024-10-14 | 419.140015  | NaN            | 175678.35217420023     |
| 2024-10-15 | 418.73999   | NaN            | 175343.1792252001      |
| 2024-10-16 | 416.119995  | 417.232001     | 173155.85023880005     |
| 2024-10-17 | 416.720001  | 417.408002     | 173655.55923344003     |
| 2024-10-18 | 418.160004  | 417.776001     | 174857.78894528002     |
| 2024-10-21 | 418.779999  | 417.703998     | 175376.68756243997     |
| 2024-10-22 | 427.51001   | 419.458002     | 182764.8086502001      |
| 2024-10-23 | 424.600006  | 421.154004     | 180285.16509520003     |
| 2024-10-24 | 424.730011